In [1]:
# if run on colab
!pip install torcheeg
#from google.colab import mount
#drive.mount('/content/drive', force_remount=True) 

In [1]:
import torcheeg
from torcheeg import transforms
from torcheeg.datasets import BCICIV2aDataset
from torcheeg.model_selection import KFoldGroupbyTrial
from torch.utils.data import DataLoader
from torcheeg.models import ATCNet, EEGNet
import torch
from torcheeg.trainers import ClassifierTrainer
import pytorch_lightning as pl

In [ ]:
root_data_path = '/kaggle/input/bci-competition-iv-dataset-2a-in-mat-format/BCICIV-2a-mat'

In [2]:
dataset = BCICIV2aDataset(
    root_path=root_data_path,
    io_path=f'./examples_pipeline/bciciv-2a',
    # skip_trial_with_artifacts=True,
    # offline_transform=transforms.Compose([
    #     transforms.BandDifferentialEntropy(apply_to_baseline=True),
    #     transforms.To2d(apply_to_baseline=True),
    #     transforms.ToTensor(apply_to_baseline=True)
    # ]),
    online_transform=transforms.Compose([
        # transforms.To2d(apply_to_baseline=True),
        # transforms.ToTensor(apply_to_baseline=True),
        transforms.To2d(),
        transforms.ToTensor(),
        # transforms.CWTSpectrum(apply_to_baseline=True),
        # transforms.BandDifferentialEntropy(apply_to_baseline=True),
        # transforms.BaselineRemoval(),
    ]),
    label_transform=transforms.Compose([
        transforms.Select('label'),
        transforms.Lambda(lambda x: x - 1)
    ]),
    chunk_size=7*250,
    num_worker=2
)

[2024-05-13 17:25:08] INFO (torcheeg/MainThread) 🔍 | Detected cached processing results, reading cache from ./examples_pipeline/bciciv-2a.


In [3]:
print("Dataset's info: ")
print(dataset.info)

Dataset's info: 

      start_at  end_at   clip_id subject_id  trial_id session subject  run  \

0          251    2001    A06E_0        A06         0       E     A06    3   

1         2254    4004    A06E_1        A06         1       E     A06    3   

2         4172    5922    A06E_2        A06         2       E     A06    3   

3         6124    7874    A06E_3        A06         3       E     A06    3   

4         8132    9882    A06E_4        A06         4       E     A06    3   

...        ...     ...       ...        ...       ...     ...     ...  ...   

5179     86751   88501  A05T_283        A05        43       T     A05    8   

5180     88657   90407  A05T_284        A05        44       T     A05    8   

5181     90585   92335  A05T_285        A05        45       T     A05    8   

5182     92699   94449  A05T_286        A05        46       T     A05    8   

5183     94758   96508  A05T_287        A05        47       T     A05    8   



      label  _record_id  

0    

In [4]:
k_fold = KFoldGroupbyTrial(
    n_splits=10,
    split_path='./examples_pipeline/split',
    shuffle=True,
    random_state=44
)

In [5]:
DEVICE = 'gpu' if torch.cuda.is_available() else 'cpu'


for i, (train_dataset, val_dataset) in enumerate(k_fold.split(dataset)):
    train_loader = DataLoader(
        dataset=train_dataset,
        batch_size=64,
        shuffle=True,
        num_workers=4
    )
    val_loader = DataLoader(
        dataset=val_dataset,
        batch_size=64,
        shuffle=False,
        num_workers=4
    )

    model = EEGNet(
        chunk_size=7*250,
        num_electrodes=22,
        num_classes=4
    )
    
    trainer = ClassifierTrainer(
        model=model,
        num_classes=4,
        lr=1e-4,
        weight_decay=1e-4,
        accelerator=DEVICE
    )

    trainer.fit(
        train_loader,
        val_loader, 
        max_epochs=50,
        default_root_dir=f'./examples_pipeline/eegnet_model/{i}',
        callbacks=[pl.callbacks.ModelCheckpoint(save_last=True)],
        enable_progress_bar=True,
        enable_model_summary=True,
        limit_val_batches=0.0
    )

    score = trainer.test(
        val_loader,
        enable_progress_bar=True,
        enable_model_summary=True
    )[0]
    print(f"Fold {i} test accuracy: {score['test_accuracy']: .4f}")

[2024-05-13 17:26:05] INFO (torcheeg/MainThread) 📊 | Detected existing split of train and test set, use existing split from ./examples_pipeline/split.

[2024-05-13 17:26:05] INFO (torcheeg/MainThread) 💡 | If the dataset is re-generated, you need to re-generate the split of the dataset instead of using the previous split.

GPU available: False, used: False

TPU available: False, using: 0 TPU cores

IPU available: False, using: 0 IPUs

HPU available: False, using: 0 HPUs

/home/quang/yes/envs/cognitive-computing/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/logger_connector/logger_connector.py:75: Starting from v1.9.0, `tensorboardX` has been removed as a dependency of the `pytorch_lightning` package, due to potential conflicts with other packages in the ML ecosystem. For this reason, `logger=True` will use `CSVLogger` as the default logger, unless the `tensorboard` or `tensorboardX` packages are found. Please `pip install lightning[extra]` or one of them to enable Te

Epoch 0: 100%|███████████████████████████████████████████████████████████████| 68/68 [00:18<00:00,  3.76it/s, v_num=0, train_loss=1.330, train_accuracy=0.250]

[2024-05-13 17:26:23] INFO (torcheeg/MainThread) 

[Train] train_loss: 1.406 train_accuracy: 0.262 



`Trainer.fit` stopped: `max_epochs=1` reached.


Epoch 0: 100%|███████████████████████████████████████████████████████████████| 68/68 [00:18<00:00,  3.75it/s, v_num=0, train_loss=1.330, train_accuracy=0.250]


GPU available: False, used: False

TPU available: False, using: 0 TPU cores

IPU available: False, using: 0 IPUs

HPU available: False, using: 0 HPUs

Missing logger folder: /home/quang/Study/Cognitive-Science/cognitive-science-final-project/lightning_logs


...

Epoch 0: 100%|███████████████████████████████████████████████████████████████| 75/75 [00:20<00:00,  3.61it/s, v_num=0, train_loss=1.080, train_accuracy=0.500]


GPU available: False, used: False

TPU available: False, using: 0 TPU cores

IPU available: False, using: 0 IPUs

HPU available: False, using: 0 HPUs


Testing DataLoader 0: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████| 7/7 [00:00<00:00, 10.91it/s]

[2024-05-13 17:29:38] INFO (torcheeg/MainThread) 

[Test] test_loss: 1.203 test_accuracy: 0.498 




Testing DataLoader 0: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████| 7/7 [00:00<00:00, 10.82it/s]

────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────

       Test metric             DataLoader 0

────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────

      test_accuracy         0.49768519401550293

        test_loss           1.2032209634780884

────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Fold 9 test accuracy:  0.4977


In [6]:
eeg_weights_path = './weights/eeg_weights.pt'
torch.save(model.state_dict(), eeg_weights_path)
# to load weight:
# model.load_state_dict(torch.load(eeg_weights_path))